In [ ]:
%html
<style>
table th, td { text-align: left; } 
</style>

# 第8章 フォーム（Form）

## 8.1 概要
### フォームの役割
- ユーザーの入力データを保持
- 入力データのバリデーション（妥当性チェック）を行い、妥当性検証済みのデータやエラーメッセージを保持

### ユーザーの入力データの保持
画面の入力フィールドの`name`属性とフォームクラスのクラス変数名を対応させて定義しておくことで、画面の入力内容からフォームオブジェクト、フォームオブジェクトから画面の入力内容へのデータ変換を容易に行うことが出来る

### 入力データのバリデーションと検証済みデータ・エラーメッセージの保持
フォームに適切な名前のバリデーションメソッドを用意しておくことで、バリデーション実行時にフォームが決まったルールに沿って、次々とバリデーションメソッドを呼び出す  
フォーム内部には、以下のようにデータが保存される
- バリデーションOK：<br />妥当性検証済みのデータ
- バリデーションNG：<br />エラーメッセージ

### フォーム利用のメリット
- ユーザーの入力データを（定義にしたがって）型変換してくれる
- バリデーションをビューから分離できる
- 入力データやエラーメッセージをテンプレートに簡単に表示できる
- ユーザー認証系のよくあるフォームはDjangoがデフォルトで用意してくれている

## 8.2 バリデーションの仕組み
### バリデーションのルール
エントリーポイント：<br />フォームの`is_valid`メソッド  
メソッドが実行されると、大まかに以下の順番でバリデーションを行う
1. フィールドに定義されたフィールド単体のバリデーション（文字種などの最低限の形式チェック）
1. フォームに定義されたフィールド単体のバリデーション（値の妥当性チェック）
1. フォームに定義された複数フィールド間のバリデーション

#### フィールドに定義されたバリデーション
- `to_python()`
- `validate()`
- `run_validators()`：<br />文字種チェックなどのバリデーションをフィールドの`validators`属性に定義している場合に実行される

#### フィールド単体に対するバリデーション
- `clean_<フィールド名>`のメソッド名で任意に用意する
- クラスで定義した順に実行する

#### 相関チェック・データベースとの整合性チェック
- `clean()`

#### 妥当性検証済みデータ
- `cleaned_data`：<br />`dict`型変数、妥当性検証済みデータを保持する<br />`is_valid()`メソッドを実行することでフォームオブジェクト内に変数を生成する

#### エラーメッセージ
- `_errors()`：<br />`list`型変数、バリデーションのエラーメッセージを保持する<br />`is_valid()`メソッドを実行することでフォームオブジェクト内に変数を生成する

## 8.3 フォームクラスの書き方
慣例的に、フォームはアプリケーションディレクトリの`forms.py`という名前のモジュールに記述する  
フォームクラスは`django.forms.Form`を継承し、内部に画面の入力フィールドに対応するクラス変数と任意のバリデーションを定義する

### 8.3.1 入力フィールドの定義
入力フィールドに対する変数には、`django.forms.fields.Field`クラスのサヴクラスを利用する
利用できる組み込みの`Field`クラスの全容は[公式ドキュメント](https://docs.djangoproject.com/ja/2.1/ref/forms/fields/#built-in-field-classes)参照

#### 利用可能な`Field`クラス（一部）

| Fieldクラス | 適用されるウィジェット | 画面に表示される入力フォールドの型 |
| :---------- | :--------------------- | :--------------------------------- |
| CharField | TextInput | テキストフィールド ｜
| EmailField | EmailInput | メールアドレスフィールド(`type="email"`) |
| BooleanField | CheckboxInput | チェックボックス |
| ChoiceField | Select | セレクトボックス |
| IntegerField | NumberInput | 数値フィールド（`type="number"`） |
| DateField | DateInput | テキストフィールド |
| TimeField | TimeInput | テキストフィールド |
| DateTimeField | DateTimeInput | テキストフィールド |
| FileField | ClearableFileInput | ファイル選択フィールド |


#### フォームクラスの例（`accounts/forms.py`）
~~~python
from django import form
from django.contrib.auth.forms import UsernameField

class LoginForm(forms.Form):
    """ログイン画面用のフォーム"""
    username = UsernameField(
        label = 'ユーザー名',
        max_length = 255,
    )
    
    password = forms.CharField(
        label = 'パスワード',
        strip = False,
        widget = forms.PasswordInput(render_value = True),
    )
~~~

#### ウィジェット（widget）
画面に表示される際の入力フィールドの型やプレースホルダ・CSSクラスなどを定義するオブジェエクト  
利用する`Field`クラスによってデフォルトで適用されるウィジェットはそれぞれ決まっている（[利用可能な`Field`クラス](#利用可能なFieldクラス（一部）)）  
デフォルトから変更したい場合は、フィールドの`widget`属性を変更する


### 8.3.2 バリデーションメソッドの定義
#### `clean_<フィールド名>`
入力値を`cleaned_data`から取得する  
バリデーションOKの場合は妥当性チェック済みの値をreturnし、`cleaned_data`に再セットする  
バリデーションNGの場合は`django.forms.ValidationError`をraiseし、フォームオブジェクト内部の値にエラーメッセージを追加する  

##### フィールド`username`のバリデーションメソッドの実装
~~~python
def clear_username(self):
    # 入力値はcleared_dataから取得
    username = self.cleared_data['username']
    # 入力チが3桁より短ければバリデーションエラー
    if len(username) < 3:
        raise forms.ValidationError('%(min_length)s文字以上で入力してください', params = {'min_length': 3})
    return username
~~~

#### `clean`メソッド
バリデーションOKの場合は、検証済みデータをreturnする必要はない  
バリデーションNGの場合は、`django.forms.ValidationError`をraiseすることで、エラーメッセージを内部の変数に追加する

##### `clean`メソッドの実装
~~~python
def clear(self):
    username = self.cleared_data['username']
    password = self.cleared_data['password']
    try:
        user = User.objects.get(username=username)
    expect ObjectDoesNotExist:
        raise forms.ValidationError("正しいユーザー名を入力してください")
    # パスワードはハッシュ化されているので、平分での検索は出来ない
    if not user.check_password(password):
       raise forms.ValidationError("正しいパスワードを入力してください")
~~~

## 8.4 ビューやテンプレートからフォームを利用する方法

- フォームは基本的に、ビュー及びテンプレート内で利用する

### 8.4.1 ビューでの利用

1. リクエストオブジェクトから入力データを取得して型変換
1. 入力データをバリデーション

#### リクエストオブジェクトから入力値を取り出す（POSTメソッドを使用しない場合）

~~~python
username = request.POST.get('username')
passowrd = request.POST.get('password')
~~~

このようにして取り出した値は全て`str`型になっている  
→型変換が必要  
バリデーションもなされていない  
→ バリデーション処理を記述する必要がある  

#### リクエストオブジェクトから入力値を取り出す（POSTメソッドを使用する場合）

~~~python
form = LoginForm(request.POST)
is_valid = form.is_valid()
~~~

フォームオブジェクトの取得とバリデーションまでが上記の2行で完結する  
`is_valid()`メソッドを呼び出すと、バリデーションがまとめて実行され、バリデーションの成否が`bool`型で返る

### 8.4.2 テンプレートでの使用例
1. テンプレート内でフォームの入力データやエラーメッセージを表示

#### フォームオブジェクトをテンプレートへレンダリングする
~~~python
{{ form }}
~~~

テンプレート内に上記の記述を行うと、以下のように入力データがセットされた入力フィールドが全て出力される  
ただし、簡易的な出力であって、CSSのスタイルなどの細かい装飾は施せない  

~~~html
<label for="id_username">ユーザー名；</label><input type="text" name="username" value="admin" id="id_username" required="">
<label for="id_password">パスワード：</label><input type="password" name="password" id="id_password" required="">
~~~

デフォルトではフォームにフィールドを記載した順に出力される  
出力順を指定する場合は、フォームに`field_order`というクラス変数を定義して、出力したいフィールドの順序を以下のように指定する

~~~python
class LoginForm(forms.Form):
    field_order = ['password', 'username']
~~~

実際は以下のようにフォームオブジェクトの要素を`for`タグでループさせてそれぞれのフィールドのスタイルを調整することが多い  

~~~html
{# --- フィールドの分だけループさせる --- #}
{% for field in form %}
<div class="field">
    {# --- 入力フィールド ---#}
    <div class="ui input">
        {{ field }}
    </div>
    {# --- 入力フィールドごとのエラーメッセージ（最初のエラーのみを表示） --- #}
    {% if field.errors %}
        <p class="red message">{{ field.errors.0 }}</p>
    {% endif %}
</div>
{% endfor %}

{# --- 全体エラーメッセージ --- #}
{% if form.non_field_errors %}
<div class="ui red message">
    <ul class="list">
        {% for non_field_error in form.non_field_errors %}
        <li>{{ non_field_error }}</li>
        {% endfor %}
    </ul>
</div>
{% endif %}
~~~

*フォームオブジェクトをイテレートすると、フォームて定義したフィールドオブジェクト（BoundField）が順番に取り出せる*

##### フォームオブジェクトのエラーについて
- フィールドに関するエラーメッセージ：<br />それぞれのフィールドの`errors`属性にリスト形式で格納
- `clean()`メソッドで追加した単体のフィールドに関連しない全体エラーメッセージ：<br />フォームオブジェクトの`non_field_errors`属性に格納

## 8.5 CSRF対策について

- Djangoでは、POSTリクエストにCSRF対策のトークン文字列が含まれていない場合に403エラーを返す仕組みがデフォルトで入っている
  - トークンは`csrfmiddlewaretoken`に入る
  
CSRF対策のトークンを設定するためには、テンプレートで`<form>`タグの内部にCSRF対策用のトークン追加のタグを記述する  
AjaxによるPOSTリクエストの場合もCSRF対策が必要  
Ajaxの場合は、HTTPヘッダに`X^CSRDToken`を設定することでも対応可能  

#### `<form>`タグの内部にCSRF対策のトークンを追加する

~~~html
<form action="" method="post" class="ui form">
…
{% csrf_token %}
</form>    
~~~

実際の表示時には以下のような`<input>`タグとして出力される  

~~~html
<form action="" method="post" class="ui form">
…
<input type="hidden" name="csrfmiddlewaretoken" value="xxxxxxxxxx">
</form>
~~~

## 8.6 ベストプラクティス7：こんなときは`ModelForm`を継承しよう
画面のフォーム作成時に、モデルとフォームのフィールド定義が似通ったときは、`django.forms.Form`の代わりに`django.forms.ModelForm`を継承してフォームを作成すると良い  
`ModelForm`を継承したフォームは、`Meta`クラスに利用するモデルクラスとフィールド名を定義しておくことで、モデルのフィールド定義を参照してフォームのフィールドへの変換が行われる

#### `ModelForm`を利用したユーザー登録画面のフォームクラスの例
~~~python
from django import forms
from django.contrib.auth.models import User

class RegisterForm(forms.ModelForm):
    """ユーザー登録画面用のフォーム"""
    class Meta:
        # 利用するモデルクラスを指定
        model = User
        # 利用するモデルのフィールドを指定
        fields = ('username', 'email', 'password')
        
…
~~~


`ModelForm`はモデルの`django.db.models.fields.Field`のサブクラスを自動判別してそれに対応するフォームの`django,forms.fields.Field`のサブクラスに変換してくれる

### モデルとフォームの`Field`クラスの対応表（一部）

| モデルの`Field`クラス | フォームの`Field`クラス |
| :-------------------: | :---------------------- |
| django.db.models.fields.CharField | django.forms.fields.CharField |
| django.db.models.fields.EmailField | django.forms.fields.EmailField |
| django.db.models.fields.BooleanField | django.forms.fields.BoleanField |
| django.db.models.fields.related.ForeignKey | django.forms.models.ModelChoiceField |
| django.db.models.fields.IntegerField | django.forms.fields.IntegerField |
| django.db.models.fields.DateField | django.forms.fields.DateField |
| django.db.models.fields.TimeField | django.forms.fields.TimeField |
| django.db.models.fields.DateTimeField | django.forms.fields.DateTimeField |
| django.db.models.fields.FileField | django.forms.fields.FileField |

### 変換されたフォームのFieldクラスそのままで使えない場合の対処
- `Meta`の`widget`でウィジェットを上書きする
- `__init__()`で`self.fields[<フィールド名>]`の属性を書き換える
- モデルに足らないフィールドの場合は、`ModelForm`クラスのフィールドを修正する

#### `ModelForm`を利用したフォームクラスの例（その２）
[`ModelForm`を利用したユーザー登録画面のフォームクラスの例](#ModelForm%E3%82%92%E5%88%A9%E7%94%A8%E3%81%97%E3%81%9F%E3%83%A6%E3%83%BC%E3%82%B6%E3%83%BC%E7%99%BB%E9%8C%B2%E7%94%BB%E9%9D%A2%E3%81%AE%E3%83%95%E3%82%A9%E3%83%BC%E3%83%A0%E3%82%AF%E3%83%A9%E3%82%B9%E3%81%AE%E4%BE%8B)に以下の修正を加えたもの
- モデルの`password`フィールドが変換された`CharField`に適用される`TextInput`ウィジェットを`PasswordInput`に変更
- `User`モデルにはない「確認用パスワード（password2）」フィールドを追加
- `email`フィールドに必須設定を付与
- 各フィールドにプレースホルダを付与

~~~python
class RegisterForm(forms.ModelForm):
    """ユーザー登録画面用のフォーム"""
    class Meta:
        model = User
        fields = ('username', 'email', 'password')
        widgets = {
            'password': forms.PasswordInput(attrs={'placeholder': 'パスワード'}),
        }
        
    password2 = forms.CharField(
        label = '確認用パスワード',
        required = True,
        strip = False,
        widget = forms.PasswordInput(attrs={'placeholder': '確認用パスワード'}),
    )
    
    def __init__(self, *args, **kwargs):
        super(RegisterForm, self).__init__(*args, **kwargs)
        self.fields['username'].widget.attrs = {'placeholder': 'ユーザー名'}
        self.fields['email'].required = True
        self.fields['email'].widget.attrs = {'placeholder': 'メールアドレス'}
~~~


### `ModelForm`が有効な場面
- モデルに定義したフィールドのうちのいくつかが画面の入力フィールドと合致する
- モデルの登録や変更を伴う

### `ModelForm`のバリデーション
1. フォームのバリデーション（文字種などの形式チェック）
1. フォームのバリデーション（値の妥当性チェック）
1. モデルのバリデーション（ユニーク制約などのデータベースとの整合性チェック）

#### `validate_unique()`メソッド
モデルの各フィールドに定義された`unique=True`の制約にしたがってレコードがユニーク担っているかどうかをチェックしてくれる  
発動条件として、フォームの`_validate_unique=True`となっていることが必要で、この変数は`ModelForm`の親クラスである`django.forms.models.BaseModelForm`の`clean()`メソッドで`True`に更新される  
$\therefore$ モデルの登録や変更を伴うフォームでは、親クラスの`clean()`メソッドを明示的に呼び出したほうがよい
$\therefore$ モデルの登録や変更を伴わないフォームでは、親クラスの`clean()`メソッドは呼び出さないほうがよい（フィールドの定義次第でユニークチェックに引っかかってしまうため）

`ModelForm`にバリデーションメソッドを記述する場合は、通常のフォームと同様に  
- `clean_<フィールド名>`：<br />フィールド単体のバリデーション
- `clean`：<br />複数フィールド間の相関チェックやデータベースとの整合性チェック
というメソッドを用意すれば良いことには変わりない

### ビューから`ModelForm`を使ったフォームを利用する
#### モデルの登録
通常のフォームと同じくリクエストオブジェクトの`GET`オブジェクトや`POST`オブジェクトをコンストラクタ引数に渡して初期化する

~~~ python
form = RegisterForm(request.POST)
~~~

`ModelForm`には`save()`メソッドが用意されており、対象のモデルをデータベースに保存することが出来る

~~~python
form.save()
~~~

POSTだけで送られてくるデータで必須情報が不足する場合は、フォームの`save()`メソッドで取得したモデルオブジェクトの`save()`メソッドを呼ぶ  
その際、フォームの`save()`メソッドに`commit=False`パラメータを渡すことで実際の登録や更新を行わないようにする  

~~~python
user = form.save(commit=False)
user.set_password(form.cleaned_data['password'])
user.save()
~~~

#### モデルの更新
以下の順序で処理する  
1. 更新したいモデルのオブジェクトをデータベースから取得
1. `instance`キーワード引数を用いてフォームのコンストラクタに渡すことで、更新前のデータをベースにしてリクエストの入力データを上書きすることが出来る

~~~python
user = User.objects.get(pk=request.user.id)
form = ProfileForm(request.post, instance=user)
~~~

## 8.7 まとめ
- フォームは`forms.py`に書く
- フォームでは入力データをクリーンにする
- フォームにはフィールド定義とバリデーションメソッドを記述する
  - フィールドには適切な`Field`クラスを使う
  - バリデーションには`clean_<フィールド名>`と`clean`という名前の2種類のメソッドを用意する
- `ModelForm`を使うことでモデルのフィールド定義を再利用することが出来る
- `POST`リクエストにはCSRF対策として`{% csrf_token %}`タグを利用する